# Hackathon Scenario 2: Multiple Vaccines and Variants
Models from early in the Covid-19 pandemic were relatively simple, as we had no vaccines available, only NPIs to mitigate the spread of the disease, and only one strain (the original wild type) to worry about. By mid-2021, the situation was much more complex, as we had access to multiple vaccines, each with multiple doses, and the emergence of multiple variants. You are interested in modeling this situation with SV2(AIR)3 (https://www.nature.com/articles/s41598- 022-06159-x, https://doi.org/10.1038/s41598-022-06159-x), one of the first models to represent multiple Covid-19 variants, and the competition between them.

1. Extract the simplest version of the model with 1 vaccine and 1 variant (shown in Fig. 1A)

  a. (TA1 Model Extraction Workflow) Extract the model from Matlab code
‘svair_simple.m’ and ‘get_beta.m’ and with the publication equations and Fig. 1A (with the exception of one small difference between code and figure). Note that in this code, the units of all state variables are in terms of fraction of the total population.
  
  b. (TA3 Simulation Workflow) Simulate the first 14 months of the pandemic beginning on January 1st, 2020. Consider only the Wild Type variant, and no vaccination during this period. Use parameter values from the supplementary material, for the wild type. Use the population of Ontario in 2019: 14.57 million people, and let the natural death rate 𝜇 = 2.05𝑒-5. For initial conditions, let 𝐼(0) = 1e-6, 𝑆(0) = 1 − 𝑠𝑢𝑚(𝐼(0) + 𝐴(0)), and all other values be 0. When do I(t) and A(t) peak, and what is the value of these variables at their peaks? How do the I(t) and A(t) profiles compare with Fig. 3d,f (which considered 3 variants and 2 types of vaccines)? Given that this question assumes only the presence of the Wild Type variant, do the results seem reasonable?
  
  c. (TA2 Domain Knowledge Grounding; ASKEM Workbench Only) Demonstrate that the domain knowledge groundings accurately reflect new concepts with the extensions such as “variants”.

In [3]:
import sympy
from mira.metamodel import *
from mira.modeling import Model
from mira.modeling.askenet.petrinet import AskeNetPetriNetModel

In [ ]:
times = [0, 82, 198, 282, 327, 370, 429, 468, 527, 548, 562, 609, 670, 731, 1096]
beta_scales = [1, 0.4, 0.55, 0.6, 0.55, 0.6, 0.55, 0.3, 0.4, 0.15, 0.2, 0.3, 0.35, 0.4, 0.5, 0.4]



"""
% simulate lockdown
function [beta_scale] = get_beta(t)
ind = 1;

% first day of simulation
day0 = '1-jan-2020';

% state of emergency and such [2]
tlock(ind)   = 82; %daysact(day0,'23-mar-2020');
tunlock(ind) = 198; %daysact(day0,'17-jul-2020');
scale(ind)   = 0.4;
ind = ind+1;

% stage 3 [3]
tlock(ind)   = 198; %daysact(day0,'17-jul-2020');
tunlock(ind) = 282; %daysact(day0,'9-oct-2020');
scale(ind)   = 0.55;
ind = ind+1;

% oops, back to stage 2 [2]
tlock(ind)   = 282; %daysact(day0,'9-oct-2020');
tunlock(ind) = 327; %daysact(day0,'23-nov-2020');
scale(ind)   = 0.6;
ind = ind+1;

% strict lockdown, stay at home order [1]
tlock(ind)   = 327; %daysact(day0,'23-nov-2020');
% tunlock(ind) = daysact(day0,'5-mar-2021');
tunlock(ind) = 370; %daysact(day0,'5-jan-2021');
scale(ind)   = 0.55;
ind = ind+1;

% stricter lockdown, stay at home order [1]
tlock(ind)   = 370; %daysact(day0,'5-jan-2021');
tunlock(ind) = 429; %daysact(day0,'5-mar-2021');
scale(ind)   = 0.3; %0.25;
ind = ind+1;

% stay at home order lifted [2]
tlock(ind)   = 429; %daysact(day0,'5-mar-2021');
tunlock(ind) = 468; %daysact(day0,'13-apr-2021');
scale(ind)   = 0.4; %0.4;
ind = ind+1;

% lockdown again [1]
tlock(ind)   = 468; %daysact(day0,'13-apr-2021');
tunlock(ind) = 527; %daysact(day0,'11-jun-2021');
scale(ind)   = 0.15;% 0.1;
ind = ind+1;

% stage 2 reopening [3]
tlock(ind)   = 527; %daysact(day0,'11-jun-2021');
tunlock(ind) = 548; %daysact(day0,'2-jul-2021');
scale(ind)   = 0.2;%0.15;
ind = ind+1;

% stage 3 reopening [4]
tlock(ind)   = 548; %daysact(day0,'2-jul-2021');
tunlock(ind) = 562; %daysact(day0,'16-jul-2021');
scale(ind)   = 0.3;%0.25;
ind = ind+1;

% physical distancing [5]
tlock(ind)   = 562; %daysact(day0,'16-jul-2021');
tunlock(ind) = 609; %daysact(day0,'1-sep-2021');
scale(ind)   = 0.35;
ind = ind+1;

% physical distancing [5]
tlock(ind)   = 609; %daysact(day0,'1-sep-2021');
tunlock(ind) = 670; %daysact(day0,'1-nov-2021');
scale(ind)   = 0.4;
ind = ind+1;

% physical distancing loosening [6]
tlock(ind)   = 670; %daysact(day0,'1-nov-2021');
tunlock(ind) = 731; %daysact(day0,'1-jan-2022');
scale(ind)   = 0.5;
ind = ind+1;

% physical distancing tightened [6]
tlock(ind)   = 731; %daysact(day0,'1-jan-2022');
tunlock(ind) = 1096; %daysact(day0,'1-jan-2023');
scale(ind)   = 0.4;
ind = ind+1;

ilock  = find(tlock<=t);
iunlock= find(t<tunlock);
ii     = intersect(ilock,iunlock);
if (length(ii)>1)
    [t,ii]
end
if (isempty(ii))
    beta_scale = 1;
else
    beta_scale = scale(ii);
end
"""

In [4]:
person_units = Unit(expression=sympy.Symbol('person'))
day_units = Unit(expression=sympy.Symbol('day'))
per_day_units = Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = Unit(expression=sympy.Integer('1'))

populations = {
    # Susceptible
    'S': [{'ido': '0000514'}, {'vaccination_status': 'vo:0001377'}],
    # Susceptible lost immunity
    'SVR': [{'ido': '0000514'}, {'disease_history': 'ido:0000592'}],
    # Vaccinated once
    'V1': [{'ido': '0000514'}, {'vaccination_status': 'askemo:0000018'}],
    # Vaccinated twice
    'V2': [{'ido': '0000514'}, {'vaccination_status': 'askemo:0000019'}],
    # Infected
    'I': [{'ido': '0000511'}, {}],
    # Infected vaccinated
    'IV': [{'ido': '0000511'}, {'vaccination_status': 'vo:0001376'}],
    # Infected after recovery
    'IR': [{'ido': '0000511'}, {'disease_history': 'ido:0000592'}],
    # Asymptomatic
    'A': [{'ido': '0000511'}, {'disease_severity': 'ncit:C3833'}],
    # Asymptomatic after recovery
    'AR': [{'ido': '0000511'}, {'disease_severity': 'ncit:C3833'}],
    # Recovered
    'R': [{'ido': '0000514'}, {'disease_severity': 'ido:0000592'}],
    # Recovered after second infection
    'R2': [{'ido': '0000592'}, {}],
}

c = {
    pop: Concept(name=pop, units=dimensionless_units,
                 identifiers=indentifiers, context=context)
    for pop, (indentifiers, context) in populations.items()
}


parameters = [
    Parameter(name='beta', value=3.3e-9, units=per_day_units),
    Parameter(name='beta_v1', value=0.2*3.3e-9, units=per_day_units),
    Parameter(name='beta_v2', value=0.05*3.3e-9, units=per_day_units),
    Parameter(name='beta_R', value=0.05*3.3e-9, units=per_day_units),
    Parameter(name='gamma', value=1/28, units=per_day_units),
    Parameter(name='mu', value=109019/(14570000*365), units=per_day_units),
    Parameter(name='mu_I', value=0.001, units=per_day_units),
    Parameter(name='mu_IV', value=0.15*0.001, units=per_day_units),
    Parameter(name='nu_R', value=1/(4*365), units=per_day_units),
    Parameter(name='nu_v1', value=1/365, units=per_day_units),
    Parameter(name='nu_v2', value=1/(4*365), units=per_day_units),
    Parameter(name='ai', value=0.5, units=dimensionless_units), # paper uses sigma instead of ai
    Parameter(name='ai_V', value=0.85, units=dimensionless_units),
    Parameter(name='ai_R', value=0.85, units=dimensionless_units),
    Parameter(name='ai_beta_ratio', value=3.0, units=dimensionless_units),
]

initials = {
    'S': Initial(concept=c['S'], value=1-1e-6),
    'SVR': Initial(concept=c['SVR'], value=0),
    'V1': Initial(concept=c['V1'], value=0),
    'V2': Initial(concept=c['V2'], value=0),
    'I': Initial(concept=c['I'], value=1e-6),
    'IV': Initial(concept=c['IV'], value=0),
    'IR': Initial(concept=c['IR'], value=0),
    'A': Initial(concept=c['A'], value=0),
    'AR': Initial(concept=c['AR'], value=0),
    'R': Initial(concept=c['R'], value=0),
    'R2': Initial(concept=c['R2'], value=0),
}

In [5]:
S, SVR, V1, V2, I, IV, IR, A, AR, R, R2 = sympy.symbols('S SVR V1 V2 I IV IR A AR R R2')

In [6]:
beta, beta_v1, beta_v2, beta_R, gamma, mu, mu_I, mu_IV, nu_R, nu_v1, nu_v2, ai, ai_V, ai_R, ai_beta_ratio = \
    sympy.symbols('beta beta_v1 beta_v2 beta_R gamma mu mu_I mu_IV nu_R nu_v1 nu_v2 ai ai_V ai_R ai_beta_ratio')

In [7]:
all_infectious = c['I'], c['IV'], c['IR'], c['A'], c['AR']
infection_rate_term = I + IV + IR + ai_beta_ratio * (A + AR)

templates = [
    # Infection asym
    GroupedControlledConversion(subject=c['S'], outcome=c['A'],
                                controllers=all_infectious,
                                rate_law=ai*beta*S*infection_rate_term),
    GroupedControlledConversion(subject=c['V1'], outcome=c['A'],
                                controllers=all_infectious,
                                rate_law=ai*beta_v1*V1*infection_rate_term),
    GroupedControlledConversion(subject=c['V2'], outcome=c['A'],
                                controllers=all_infectious,
                                rate_law=ai*beta_v2*V2*infection_rate_term),

    # Infection sym
    GroupedControlledConversion(subject=c['S'], outcome=c['I'],
                                controllers=all_infectious,
                                rate_law=(1-ai)*beta*S*infection_rate_term),
    GroupedControlledConversion(subject=c['V1'], outcome=c['IV'],
                                controllers=all_infectious,
                                rate_law=ai*beta_v1*V1*infection_rate_term),
    GroupedControlledConversion(subject=c['V2'], outcome=c['IV'],
                                controllers=all_infectious,
                                rate_law=ai*beta_v2*V2*infection_rate_term),
    # Vaccination
    # Note: structurally we want to have these in the model but we make the rates 0
    NaturalConversion(subject=c['S'], outcome=c['V1'], rate_law=0*S),
    NaturalConversion(subject=c['A'], outcome=c['V1'], rate_law=0*A),
    NaturalConversion(subject=c['V1'], outcome=c['V2'], rate_law=0*V1),
    # Recovery
    NaturalConversion(subject=c['I'], outcome=c['R'], rate_law=gamma*I),
    NaturalConversion(subject=c['IV'], outcome=c['R'], rate_law=gamma*IV),
    NaturalConversion(subject=c['IR'], outcome=c['R'], rate_law=gamma*IR),
    NaturalConversion(subject=c['A'], outcome=c['R'], rate_law=gamma*A),
    NaturalConversion(subject=c['AR'], outcome=c['R'], rate_law=gamma*AR),

    # Reinfection asym
    GroupedControlledConversion(subject=c['R'], outcome=c['AR'],
                                controllers=all_infectious,
                                rate_law=ai_R*beta_R*R*infection_rate_term),
    # Reinfection sym
    GroupedControlledConversion(subject=c['R'], outcome=c['IR'],
                                controllers=all_infectious,
                                rate_law=(1-ai_R)*beta_R*R*infection_rate_term),

    # Second recovery
    NaturalConversion(subject=c['IR'], outcome=c['R2'], rate_law=gamma*IR),
    NaturalConversion(subject=c['AR'], outcome=c['R2'], rate_law=gamma*AR),

    # Death
    NaturalDegradation(subject=c['S'], rate_law=mu*S),
    NaturalDegradation(subject=c['SVR'], rate_law=mu*SVR),
    NaturalDegradation(subject=c['V1'], rate_law=mu*V1),
    NaturalDegradation(subject=c['V2'], rate_law=mu*V2),
    NaturalDegradation(subject=c['I'], rate_law=mu_I*I),
    NaturalDegradation(subject=c['IV'], rate_law=mu_IV*IV),
    NaturalDegradation(subject=c['IR'], rate_law=mu*IR),
    NaturalDegradation(subject=c['A'], rate_law=mu*A),
    NaturalDegradation(subject=c['AR'], rate_law=mu*AR),
    NaturalDegradation(subject=c['R'], rate_law=mu*R),
    NaturalDegradation(subject=c['R2'], rate_law=mu*R2),

    # Loss of immunity
    NaturalConversion(subject=c['V1'], outcome=c['SVR'], rate_law=nu_v1*V1),
    NaturalConversion(subject=c['V2'], outcome=c['SVR'], rate_law=nu_v2*V2),
    NaturalConversion(subject=c['R'], outcome=c['SVR'], rate_law=nu_R*R),
    NaturalConversion(subject=c['R2'], outcome=c['SVR'], rate_law=nu_R*R2),
]

In [8]:
tm = TemplateModel(templates=templates,
                   parameters={p.name: p for p in parameters},
                   initial_conditions=initials)
tm = simplify_rate_laws(tm)
tm.annotations= Annotations(name="Scenario 2a")
AskeNetPetriNetModel(Model(tm)).to_json_file('scenario2_a.json')